# Goal: 
    * 1. Unzip monthly tripdata 
    * 2. Creating a DataFrame with every trip in 2022
    * 3. Cleaning Data/ changing datatypes
    * 4. Push to sql database/store as csv on local machine

Datasource:
https://divvy-tripdata.s3.amazonaws.com/index.html

## Importing needed Packages

In [1]:
# Import libraries
import pandas as pd
import zipfile
import os

# Import libraries for SQL
from sql_functions import get_engine

## 1. Unzip monthly tripdata 

In [26]:
# constants:
path = "data/"
schema = "capstone_divvy_bikeshare"
engine = get_engine()

In [3]:
# checking data folder for specific zip files from year 2022 and creating a list of filenames:
list_tripfiles_2022 = []
for file in os.listdir(path):
    if file.startswith("2022") and file.endswith("tripdata.zip") == True:
        list_tripfiles_2022.append(file)
list_tripfiles_2022

['202206-divvy-tripdata.zip',
 '202208-divvy-tripdata.zip',
 '202205-divvy-tripdata.zip',
 '202203-divvy-tripdata.zip',
 '202210-divvy-tripdata.zip',
 '202211-divvy-tripdata.zip',
 '202201-divvy-tripdata.zip',
 '202202-divvy-tripdata.zip',
 '202212-divvy-tripdata.zip',
 '202209-divvy-tripdata.zip',
 '202204-divvy-tripdata.zip',
 '202207-divvy-tripdata.zip']

In [4]:
# taking a look at january:
zf_jan22 = zipfile.ZipFile("data/202201-divvy-tripdata.zip")
df = pd.read_csv(zf_jan22.open("202201-divvy-tripdata.csv"))
df.head()

ride_id  rideable_type           started_at             ended_at  \
0  C2F7DD78E82EC875  electric_bike  2022-01-13 11:59:47  2022-01-13 12:02:44   
1  A6CF8980A652D272  electric_bike  2022-01-10 08:41:56  2022-01-10 08:46:17   
2  BD0F91DFF741C66D   classic_bike  2022-01-25 04:53:40  2022-01-25 04:58:01   
3  CBB80ED419105406   classic_bike  2022-01-04 00:18:04  2022-01-04 00:33:00   
4  DDC963BFDDA51EEA   classic_bike  2022-01-20 01:31:10  2022-01-20 01:37:12   

              start_station_name start_station_id  \
0       Glenwood Ave & Touhy Ave              525   
1       Glenwood Ave & Touhy Ave              525   
2  Sheffield Ave & Fullerton Ave     TA1306000016   
3       Clark St & Bryn Mawr Ave     KA1504000151   
4    Michigan Ave & Jackson Blvd     TA1309000002   

                end_station_name end_station_id  start_lat  start_lng  \
0           Clark St & Touhy Ave         RP-007  42.012800 -87.665906   
1           Clark St & Touhy Ave         RP-007  42.012763 -87.665967   
2  Greenview Ave & Fullerton Ave   TA1307000001  41.925602 -87.653708   
3      Paulina St & Montrose Ave   TA1309000021  41.983593 -87.669154   
4         State St & Randolph St   TA1305000029  41.877850 -87.624080   

     end_lat    end_lng member_casual  
0  42.012560 -87.674367        casual  
1  42.012560 -87.674367        casual  
2  41.925330 -87.665800        member  
3  41.961507 -87.671387        casual  
4  41.884621 -87.627834        member

In [14]:
# creating a function, to extract zipfiles, and store them in a dataframe:
def zip_to_df(file):
    # # exception: The zip file for september 2022 has a sightly differently called csv file:
    if file == "202209-divvy-tripdata.zip":
        zf = zipfile.ZipFile(path + "202209-divvy-tripdata.zip")
        df = pd.read_csv(zf.open("202209-divvy-publictripdata" + ".csv"))
    else:
        zf = zipfile.ZipFile(path + file)
        df = pd.read_csv(zf.open(file[:-4] + ".csv"))
    return df



Testing zip_to_df function:

In [15]:
zip_to_df("202209-divvy-tripdata.zip").head()

ride_id  rideable_type           started_at             ended_at  \
0  5156990AC19CA285  electric_bike  2022-09-01 08:36:22  2022-09-01 08:39:05   
1  E12D4A16BF51C274  electric_bike  2022-09-01 17:11:29  2022-09-01 17:14:45   
2  A02B53CD7DB72DD7  electric_bike  2022-09-01 17:15:50  2022-09-01 17:16:12   
3  C82E05FEE872DF11  electric_bike  2022-09-01 09:00:28  2022-09-01 09:10:32   
4  4DEEB4550A266AE1  electric_bike  2022-09-01 07:30:11  2022-09-01 07:32:36   

  start_station_name start_station_id                end_station_name  \
0                NaN              NaN  California Ave & Milwaukee Ave   
1                NaN              NaN                             NaN   
2                NaN              NaN                             NaN   
3                NaN              NaN                             NaN   
4                NaN              NaN                             NaN   

  end_station_id  start_lat  start_lng    end_lat    end_lng member_casual  
0          13084      41.93     -87.69  41.922695 -87.697153        casual  
1            NaN      41.87     -87.62  41.870000 -87.620000        casual  
2            NaN      41.87     -87.62  41.870000 -87.620000        casual  
3            NaN      41.93     -87.69  41.940000 -87.670000        casual  
4            NaN      41.92     -87.73  41.920000 -87.730000        casual

## 2. Creating a DataFrame with every trip in 2022

Now lets load all 12 zipfiles (one for every month of 2022) into a dataframe, and then concat them into one big df_22 dataframe:

In [16]:
# converting zipfile of every month into dataframes:
for file in list_tripfiles_2022:
    if "202201" in file:
        df_01 = zip_to_df(file)
    elif "202202" in file:
        df_02 = zip_to_df(file)
    elif "202203" in file:
        df_03 = zip_to_df(file)
    elif "202204" in file:
        df_04 = zip_to_df(file)
    elif "202205" in file:
        df_05 = zip_to_df(file)
    elif "202206" in file:
        df_06 = zip_to_df(file)
    elif "202207" in file:
        df_07 = zip_to_df(file)
    elif "202208" in file:
        df_08 = zip_to_df(file)
    elif "202209" in file:
        df_09 = zip_to_df(file)
    elif "202210" in file:
        df_10 = zip_to_df(file)
    elif "202211" in file:
        df_11 = zip_to_df(file)
    elif "202212" in file:
        df_12 = zip_to_df(file)



In [18]:
# Concatinating monthly dataframes into a one year dataframe called df_22:
df_22 = pd.concat([df_01, df_02, df_03, df_04, df_05, df_06, df_07, df_08, df_09, df_10, df_11, df_12], axis = 0)

## 3. Cleaning Data/ changing datatypes...

Creating a function to clean the DataFrame:

In [21]:
def clean_df(df):
    df.rename(columns = {'started_at':'starttime', 'ended_at':'stoptime', 'start_station_name':'from_station_name', 'start_station_id':'from_station_id', 'end_station_name':'to_station_name', 'end_station_id':'to_station_id'}, inplace = True)
    df['starttime'] = pd.to_datetime(df['starttime'])
    df['stoptime'] = pd.to_datetime(df['stoptime'])
    df['from_station_id'].fillna(0, inplace=True)
    df['from_station_id'] = df['from_station_id'].astype('str')
    df['from_station_id'] = df['from_station_id'].str.replace(".0", "")
    df['from_station_id'] = df['from_station_id'].str.replace(",", "")
    df['to_station_id'].fillna(0, inplace=True)
    df['to_station_id'] = df['to_station_id'].astype('str')
    df['to_station_id'] = df['to_station_id'].str.replace(".0", "")
    df['to_station_id'] = df['to_station_id'].str.replace(",", "")
    df.drop_duplicates(inplace=True)
    df.sort_values('starttime', inplace=True)
    df.reset_index(drop=True, inplace=True)
    # copy columns to a rearranged DataFrame
    df_final = df[['ride_id', 'rideable_type', 'starttime', 'stoptime', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual']].copy()
    return df_final

Cleaning the DataFrame:

In [22]:
df_22 = clean_df(df_22)

/var/folders/rb/9ddkw0p569d7d8xgjzc89f_c0000gn/T/ipykernel_13145/3306187110.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['from_station_id'] = df['from_station_id'].str.replace(".0", "")
/var/folders/rb/9ddkw0p569d7d8xgjzc89f_c0000gn/T/ipykernel_13145/3306187110.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['to_station_id'] = df['to_station_id'].str.replace(".0", "")


In [24]:
df_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5667717 entries, 0 to 5667716
Data columns (total 13 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ride_id            object        
 1   rideable_type      object        
 2   starttime          datetime64[ns]
 3   stoptime           datetime64[ns]
 4   from_station_id    object        
 5   from_station_name  object        
 6   to_station_id      object        
 7   to_station_name    object        
 8   start_lat          float64       
 9   start_lng          float64       
 10  end_lat            float64       
 11  end_lng            float64       
 12  member_casual      object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 562.1+ MB


## 4. Push to sql database/store as csv on local machine

In [28]:
# #Push DataFrame to SQL Database:
# table_name = 'trips_20xx'

# df_22.to_sql(name=table_name, # Name of SQL table
#                     con=engine, # Engine or connection
#                     if_exists='replace', # Drop the table before inserting new values 
#                     schema=schema, # Use schema that was defined earlier
#                     index=False, # Write DataFrame index as a column
#                     chunksize=5000, # Specify the number of rows in each batch to be written at a time
#                     method='multi') # Pass multiple values in a single INSERT clause
# print(f"The {table_name} table was imported successfully.")

The trips_2022_test table was imported successfully.
